In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#Mounting Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import pandas as pd 
df = pd.read_csv('../Resources/Debernardi et al 2020 data.csv')
df.head()

,sample_id,patient_cohort,sample_origin,age,sex,diagnosis,stage,benign_sample_diagnosis,plasma_CA19_9,creatinine,LYVE1,REG1B,TFF1,REG1A
0,S1,Cohort1,BPTB,33,F,1,NaN,NaN,11.7,1.83222,0.893219,52.94884,654.282174,1262.000
1,S10,Cohort1,BPTB,81,F,1,NaN,NaN,NaN,0.97266,2.037585,94.46703,209.488250,228.407
2,S100,Cohort2,BPTB,51,M,1,NaN,NaN,7.0,0.78039,0.145589,102.36600,461.141000,NaN
3,S101,Cohort2,BPTB,61,M,1,NaN,NaN,8.0,0.70122,0.002805,60.57900,142.950000,NaN
4,S102,Cohort2,BPTB,62,M,1,NaN,NaN,9.0,0.21489,0.000860,65.54000,41.088000,NaN


In [3]:
df = df.drop(["sample_id", "patient_cohort", "sample_origin", "stage", "benign_sample_diagnosis", "plasma_CA19_9", "creatinine" ], 1)
df

C:\Users\begar\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,age,sex,diagnosis,LYVE1,REG1B,TFF1,REG1A
0,33,F,1,0.893219,52.948840,654.282174,1262.000
1,81,F,1,2.037585,94.467030,209.488250,228.407
2,51,M,1,0.145589,102.366000,461.141000,NaN
3,61,M,1,0.002805,60.579000,142.950000,NaN
4,62,M,1,0.000860,65.540000,41.088000,NaN
...,...,...,...,...,...,...,...
585,68,M,3,7.058209,156.241000,525.178000,NaN
586,71,F,3,8.341207,16.915000,245.947000,NaN
587,63,M,3,7.674707,289.701000,537.286000,NaN
588,75,F,3,8.206777,205.930000,722.523000,NaN


In [4]:
df.nunique()

age           63
sex            2
diagnosis      3
LYVE1        535
REG1B        580
TFF1         560
REG1A        298
dtype: int64

In [5]:
# df = df.fillna(0)
df= df.dropna(subset=['REG1A'])

In [25]:
# dummies_df = pd.get_dummies(df)
# dummies_df

In [6]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Generate categorical variables
cat_var = df.dtypes[df.dtypes == "object"].index.tolist()
print(cat_var)

['sex']


In [7]:
enc = OneHotEncoder(sparse = False)

encode_df = pd.DataFrame(enc.fit_transform(df[cat_var]))

encode_df.columns = enc.get_feature_names(cat_var)
encode_df.head()

C:\Users\begar\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,sex_F,sex_M
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [8]:
df = df.merge(encode_df, left_index=True, right_index=True)
df = df.drop(cat_var, axis=1)
df.head()

,age,diagnosis,LYVE1,REG1B,TFF1,REG1A,sex_F,sex_M
0,33,1,0.893219,52.94884,654.282174,1262.000,1.0,0.0
1,81,1,2.037585,94.46703,209.488250,228.407,1.0,0.0
12,49,1,1.416314,151.83077,74.189903,505.571,0.0,1.0
19,28,1,0.583010,13.61906,267.193539,381.000,1.0,0.0
20,54,1,0.004976,5.50735,193.145661,113.000,1.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = df.drop(["diagnosis"], axis="columns").values
# X = X.fillna(0)
y = df["diagnosis"].values


# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
from tensorflow.keras.layers import BatchNormalization

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
number_input_features = len(X_train_scaled[0])
# model.add(tf.keras.layers.Dense(256, input_shape=(x_train.shape[1],), activation='sigmoid'))
# hidden_nodes_layer1 = 150
# hidden_nodes_layer2 = 100
# hidden_nodes_layer3 = 80
# hidden_nodes_layer4 = 30


nn_model = tf.keras.models.Sequential()

# First hidden layer
# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=number_input_features))

# Second hidden layer
#  YOUR CODE GOES HERE
nn_model.add(
    tf.keras.layers.Dense(units=50, activation="relu")
)

#  YOUR CODE GOES HERE
# nn_model.add(
#     tf.keras.layers.Dense(units=1, activation="sigmoid")
# )
nn_model.add(
    tf.keras.layers.Dense(units=1, activation="softmax")
)

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 40        
                                                                 
 dense_1 (Dense)             (None, 50)                300       
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 391
Trainable params: 391
Non-trainable params: 0
_________________________________________________________________


In [12]:
# nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
3/3 [==============================] - 1s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 2/50
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 3/50
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 4/50
3/3 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 5/50
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 6/50
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 7/50
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 8/50
3/3 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 9/50
3/3 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.6702
Epoch 10/50
3/3 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.6702

In [14]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.0000e+00 - accuracy: 0.5000 - 493ms/epoch - 493ms/step
Loss: 0.0, Accuracy: 0.5


In [16]:
classifier.fit(X_train, y_train)

NameError: name 'classifier' is not defined

In [15]:
from sklearn.metrics import confusion_matrix, classification_report
# y_pred = classifier.predict(X_test)
y_pred = classifier.predict(X_test_scaled)
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

NameError: name 'classifier' is not defined